In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tabulate import tabulate
import numpy as np

!pip install openai


First we use OpenAi API to generate different sentence each entailing a different command: 1. Telling stories 2. Seeking advice and guidance 3. Causal chat

In [3]:
# This code is for v1 of the openai package: pypi.org/project/openai
from openai import OpenAI
import time
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

CLASS_DATA_NUM = [20, 20, 20]

testSentences = []
testLabels = []

for cat in range(3):


  message = str(cat + 1)
  for i in range(CLASS_DATA_NUM[cat]):
    response = client.chat.completions.create(
    model="gpt-4",
    messages=[
      {
        "role": "system",
        "content": "We want to build a classifier which is able to categorize the input into three categories. The inputs are sentences written by a school-age child who is talking to a chatbot. We want three classes of sentences:\n1. The sentences in which the kid is implying or asking the chatbot to tell him/her a story. \n2. The sentences which are implying that the child needs help, guidance or advice.\n3. The sentences which are implying that the child wants to have a casual chat.\nIt is important to note that the child might not directly ask for a piece of advice or a guidance or a story.  For example, the sentence: 'It would be great if I knew how to play basketball' is categorized in the second class because it is clear that by giving him some advice, we can help him. On the other hand, if the child is expressing his or her problem or negative feeling or difficulty without any sign or indication of seeking help or comment, that sentence belongs the third class. Therefore, building such a dataset requires realistic thinking and subtleties to avoid generating only trivial and obvious sentences.\n\nYour task is given one of values: {1, 2, 3}, generate a sentence which belongs to that class and is said by a hypothetical school-age child keeping in mind the details and instructions that were made in this text earlier.\nTry to make more sophisticated sentences with realistic details. Try not to generate cliche sentences.\n\nExample:\n\nInput : 1\nOutput: \"I'm a little bored. I really want to hear an exciting story\"\n\nInput : 2\nOutput: \"I really can't tolerate this mean bully! It would be great if someone could help me deal with him\"\n\nInput : 3\n\nOutput: \"I really liked what she said about me but I'm still angry at her.\"\n "
      },
      {
        "role": "user",
        "content": message
      }
      ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )

    testSentences.append(response.choices[0].message.content)

    time.sleep(1)









In [25]:

testSentences = []
testLabels = []
for cat in range(3):
  for i in range(CLASS_DATA_NUM[cat]):
    testLabels.append(cat)


with open('commandTestSentences.txt','r') as f:
  testSentences = f.readlines()

In [26]:
manualSentences = ["I am really in mood of hearing a story!",
                   "Can you tell me a story before I go to bed?",
                   "Tell me a fairytale which I've never heard.",
                   "I want to tell my little sister a story but I don't know any!",
                   "What is the shortest story you know?",
                   "My mom always tells me the same tale. I want to hear a new one.",


                   "Jerry did some really amazing magic tricks today. I wish I could too",
                   "I really don't know how to deal with my teammate. He doesn't do anything.",
                   "I wanna surprise Mom. How can I make her happy when she gets home?",
                   "My teacher told me not to waste my time with video games, but how?!",
                   "Do you know how to do substractions?",
                   "What can I do to get A+ in my science class?",
                   "Is it mean to tell make fun of Ashley after what she said about me today?",

                   "Mr. Abrams is really cool. I wish all teacher were like him.",
                   "I'm really pissed at John. He made fun of me in front of the whole class.",
                   "Today was fun. But tomorrow will be boring as hell.",
                   "What a lovely puppy!",
                   "Sometime Mom gets so annoying. Why can't I have a baby cat?",
                   "I got a C in math. I let everyone down.",
                   "I would love to watch the new Snow white movie!"]

manualLabels = [0,0,0,0,0,0, 1,1,1,1,1,1,1, 2,2,2,2,2,2,2]

for i in range(len(manualSentences)):
  testSentences.append(manualSentences[i])
  testLabels.append(manualLabels[i])


In [11]:
print([testSentences[5]])

['"Can you take me to a magical world? You know, with fairies, dragons, and knights? Something like a story would be great!"\n']


In [ ]:
import csv

with open('commandTestSentences.txt','w') as f:
  for sentence in testSentences:
    f.write(sentence)
    f.write('\n')

In [29]:

corr = 0
outputs = []
for sentence in testSentences:
  response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "system",
      "content": "You are given a sentence which is written by a school-age kid. You are a chatbot and your task is to figure out when the child's message implies that they want to hear a story, get some advice and guidance from you, or just want to casually chat and talk about the emotion they are going through. If children want to receive advice or guidance, they either say this directly or they might tell you that they have a problem that they need to face. They may ask you to help them with some homework or some other issue. Sometimes they ask you to tell them a story. Sometimes, based on what they say, they do not particularly want to chat casually and talk about their day, their feelings, their thoughts, etc. \nSometimes you may confuse whether the user who is expressing a difficulty or a problem, seeks advice or guidance or just wants to talk about their problem. In this case you have to ask for clarification. The child does not seek some advice unless Based on the user message, you have to find out the user's preference and output it.\nIf the user wants to hear a story, output 0.\nIf the user wants to receive advice and guidance, output 1.\nIf the user wants to have a casual conversation with no particular purpose or , output 2.\nIf you are not sure about user's intent and you have to ask some clarification, output 3.\n\n Explain your decision. Output the class number and then your explanation: \n\nExample:\n\nInput: Alice is very mean to me. I can't tolerate her anymore.\n\nOutput: \n 3\n\nThe user is clearly expressing a difficulty or a problem, but we are not sure whether she seeks some advice or just want to talk about her day and her feelings. Therefore, we need to ask for clarification.\n\nExample:\nInput:\n It would be great if I knew how to play basketball.\n\nOutput:\n 1\n\nThe user explicitly expresses his intent to learn basketball. Therefore, there is no doubt that by giving him some guidance we can help him.\n\n\nNote: When users are expressing a problem or a difficulty, if they are also looking for a solution or way to deal with it, we can confidently decide that the output is 1, otherwise we need to let conversation continues to clarify the user's purpose , thus output 3\n"
    },
   {
      "role": "user",
      "content": sentence
   }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
  )

  print(response.choices[0].message.content)
  outputs.append(int(response.choices[0].message.content.strip()[0]))

  time.sleep(5)







0

The user is asking for a story, specifically one about a brave knight who saves the day. So, it's clear that they want to hear a story.
0

The user expresses interest in stories specifically about knights and dragons and invites the chatbot to share one. It is clear that the child wants to hear a story, thus, the output is 0.
0

The child is asking for another story similar to the one they heard before about a dragon and a princess. There is no mention of a problem they need help with, nor is there a casual chat about their feelings or experiences. The request is clearly for a story, hence the output is 0.
0

The child is asking for a fun tale to pass the long evenings, indicating that they want to hear a story.
0

The user is asking for a fairy tale, specifically about a brave knight and a dragon. This clearly indicates the user wants to hear a story.
0

The user is expressing interest in the idea of a magical world with fairies, dragons, and knights. Given they mentioned that "som

In [40]:
cor = 0

idx = []
i = 0
for (pred, label) in zip(outputs, testLabels):
  if pred == label:
    cor +=1
  else:
    idx.append(i)

  i += 1

print(idx)

j = 78
print(testSentences[j],outputs[j])

print(testLabels[j])




[41, 44, 46, 49, 51, 53, 59, 74, 77, 78]
I got a C in math. I let everyone down.
 1
2


In [23]:
len(testLabels)

40


"Mary is angry. She wants a piece of advice.